In [1]:
import fetch from "npm:node-fetch"
//import * as fs from "https://deno.land/std@0.220.1/fs/mod.ts";
import * as fs from "https://deno.land/std@0.143.0/node/fs.ts";

In [5]:
// library to make HTTP calls
// const fetch = require('node-fetch');
// library to write to the filesystem
// const fs = require('fs');
  
//util method to write to the filesystem
const downloadFile = (async (url, path) => {
  const res = await fetch(url);
  const fileStream = fs.createWriteStream(path);
  await new Promise((resolve, reject) => {
    res.body.pipe(fileStream);
    res.body.on("error", reject);
    fileStream.on("finish", resolve);
  });
});
  
//last time program was run (you should change this value or make it a parameter of your script)
const lastCheck = '2023-03-14T18:00';
// online data code to lookup (you should change this value or make it a parameter of your script)
const onlineDataCode = "TPS00001";
  
(async () => {
  //retrieve information on dataset in JSON
  //console.log('https://ec.europa.eu/eurostat/api/dissemination/sdmx/2.1/dataflow/ESTAT/' + onlineDataCode + '?format=json')
  const response = await fetch('https://ec.europa.eu/eurostat/api/dissemination/sdmx/2.1/dataflow/ESTAT/' + onlineDataCode + '?format=json');
  const json = await response.json();
    
  //extract last data udpate date
  var lastUpdateDate = null;
  for (var i = 0 ; i < json.extension.annotation.length; i++) {
    console.log(json.extension.annotation[i]);
    if (json.extension.annotation[i].type == "UPDATE_DATA")
      lastUpdateDate = json.extension.annotation[i].date;
    }
    
  //compare last data update date with our own
  if (Date.parse(lastUpdateDate) > Date.parse(lastCheck)) {
    console.log(onlineDataCode + " need update " + " last updated at " + lastUpdateDate);
    //download needed format (remove the line you do not need)
    console.log("Downloading data in TSV for " + onlineDataCode );
    downloadFile('https://ec.europa.eu/eurostat/api/dissemination/sdmx/2.1/data/' + onlineDataCode + '?format=TSV', onlineDataCode +'.tsv');
    console.log("Downloading data in SDMX-CSV for " + onlineDataCode );
    downloadFile('https://ec.europa.eu/eurostat/api/dissemination/sdmx/2.1/data/' + onlineDataCode + '?format=SDMX-CSV', onlineDataCode +'.sdmx.csv');
    console.log("Downloading data in SDMX-ML 2.1 for " + onlineDataCode );
    downloadFile('https://ec.europa.eu/eurostat/api/dissemination/sdmx/2.1/data/' + onlineDataCode , onlineDataCode +'.sdmx.xml');
    console.log("Downloading structure in SDMX-ML 2.1 for " + onlineDataCode );
    downloadFile('https://ec.europa.eu/eurostat/api/dissemination/sdmx/2.1/dataflow/ESTAT/' + onlineDataCode +'?references=descendants&details=referencepartial', onlineDataCode +'.sdmx.structure.xml');
 
    // or extract specific time-series in json-stat to feed visualisations..
    downloadFile('https://ec.europa.eu/eurostat/api/dissemination/sdmx/2.1/data/' + onlineDataCode + '/A.....FR+LU+DE?format=JSON', onlineDataCode +'_FR_LU_DE.json');
    // or use Eurostat statistics query format, because SDMX REST have may be less readable depending the query (may have too many dots, need to know the order of dims)
    downloadFile('https://ec.europa.eu/eurostat/api/dissemination/statistics/1.0/data/' + onlineDataCode + '?geo=FR&geo=LU&geo=DE', onlineDataCode +'_FR_LU_DE.stats.json');
  } else {
    console.log(onlineDataCode + " already up to date " + " last updated at " + lastUpdateDate);
  }
  
})();

{ type: "DISSEMINATION_OBJECT_TYPE", title: "DATASET" }
{ type: "DISSEMINATION_SOURCE_DATASET", title: "DEMO_GIND" }
{ type: "OBS_COUNT", title: "589" }
{ type: "OBS_PERIOD_OVERALL_OLDEST", title: "2012" }
{ type: "OBS_PERIOD_OVERALL_LATEST", title: "2023" }
{ type: "UPDATE_STRUCTURE", date: "2023-12-22T11:00:00+0100" }
{ type: "UPDATE_DATA", date: "2023-12-22T11:00:00+0100" }
{ type: "SOURCE_INSTITUTIONS", text: "Eurostat" }
{
  type: "DISSEMINATION_DOI_XML",
  title: '<adms:identifier xmlns:adms="http://www.w3.org/ns/adms#" xmlns:skos="http://www.w3.org/2004/02/skos/'... 486 more characters
}
{
  type: "DISSEMINATION_PRESENTATIONS",
  title: '[{"names":{"de":{"locale":"de","value":"Standardpräsentation","html":false},"en":{"locale":"en","val'... 832 more characters
}
{ type: "DISSEMINATION_PRESENTATIONS_VERSION", title: "v0_4_X" }
TPS00001 need update  last updated at 2023-12-22T11:00:00+0100


Promise { <pending> }